In [ ]:
# === 1. Game State Logic ===
from copy import deepcopy
import pandas as pd

NUM_ROWS = 6
NUM_COLS = 7

class State:
    def __init__(self):
        self.board = [[0]*NUM_COLS for _ in range(NUM_ROWS)]
        self.column_heights = [NUM_ROWS - 1] * NUM_COLS
        self.available_moves = list(range(7))
        self.player = 1
        self.winner = -1

    def check_line(self, n, player, values):
        num_pieces = sum(val == player for val in values)
        if n == 4:
            return num_pieces == 4
        if n == 3:
            return num_pieces == 3 and values.count(0) == 1

    def count_lines(self, n, player):
        num_lines = 0
        for row in range(NUM_ROWS):
            for col in range(NUM_COLS):
                if col < NUM_COLS - 3 and self.check_line(n, player, [self.board[row][col+i] for i in range(4)]):
                    num_lines += 1
                if row < NUM_ROWS - 3 and self.check_line(n, player, [self.board[row+i][col] for i in range(4)]):
                    num_lines += 1
                if row < NUM_ROWS - 3 and col < NUM_COLS - 3 and self.check_line(n, player, [self.board[row+i][col+i] for i in range(4)]):
                    num_lines += 1
                if row < NUM_ROWS - 3 and col > 2 and self.check_line(n, player, [self.board[row+i][col-i] for i in range(4)]):
                    num_lines += 1
        return num_lines

    def central(self, player):
        points = 0
        for row in range(NUM_ROWS):
            if self.board[row][4] == player:
                points += 2
            if self.board[row][3] == player or self.board[row][5] == player:
                points += 1
        return points

    def move(self, column):
        state_copy = deepcopy(self)
        height = state_copy.column_heights[column]
        state_copy.board[height][column] = self.player
        if height == 0:
            state_copy.available_moves.remove(column)
        else:
            state_copy.column_heights[column] = height - 1
        state_copy.update_winner()
        state_copy.player = 3 - self.player
        return state_copy

    def update_winner(self):
        if self.count_lines(4, 1) > 0:
            self.winner = 1
        elif self.count_lines(4, 2) > 0:
            self.winner = 2
        elif not self.available_moves:
            self.winner = 0

In [2]:
# === 2. Game Interface ===
def display_board(board):
    print("\n  1 2 3 4 5 6 7")
    print(" ---------------")
    for row in board:
        print("|", end="")
        for cell in row:
            print(" ".join("X" if cell == 1 else "O" if cell == 2 else " "), end="|")
        print()
    print(" ---------------")

def print_welcome():
    print("\n===== CONNECT 4 =====")
    print("1. Player vs Player")
    print("2. Player vs Computer")
    print("3. Computer vs Computer")
    print("4. Exit")
    print("====================")



In [3]:
# === 3. Game Controller Class ===
import time

class ConnectFourGame:
    def __init__(self, player_1_ai, player_2_ai):
        self.state = State()
        self.player_1_ai = player_1_ai
        self.player_2_ai = player_2_ai
        self.match_log = []

    def record_state(self):
        flat_board = sum(self.state.board, [])
        state_record = {f'cell_{i}': val for i, val in enumerate(flat_board)}
        state_record['player'] = self.state.player
        state_record['winner'] = self.state.winner
        self.match_log.append(state_record)

    def save_match_log(self, filename="match_log.csv"):
        if self.match_log:
            df = pd.DataFrame(self.match_log)
            df.to_csv(filename, mode='a', header=not pd.io.common.file_exists(filename), index=False)
            self.match_log = []

    def start(self, log_moves=False):
        self.state = State()
        while self.state.winner == -1:
            self.record_state()
            if self.state.player == 1:
                self.player_1_ai(self)
            else:
                self.player_2_ai(self)
            if log_moves:
                display_board(self.state.board)
        self.record_state()
        self.save_match_log()
        if self.state.winner == 0:
            print("End of game! Draw!")
        else:
            print(f"End of game! Player {self.state.winner} wins!")

    def run_n_matches(self, n, max_time=3600, log_moves=False):
        start_time = time.time()
        results = [0, 0, 0]
        while n > 0 and time.time() - start_time < max_time:
            n -= 1
            self.start(log_moves)
            results[self.state.winner] += 1
        print("\n=== Elapsed time: %s seconds ===" % int(time.time() - start_time))
        print(f"  Player 1: {results[1]} victories")
        print(f"  Player 2: {results[2]} victories")
        print(f"  Draws: {results[0]} ")
        print("===============================")

In [4]:
# === 4. AI Players ===
import random

def execute_random_move(game):
    move = random.choice(game.state.available_moves)
    game.state = game.state.move(move)

def execute_smart_move(game):
    available_moves = game.state.available_moves
    current_player = game.state.player
    opponent = 3 - current_player

    for move in available_moves:
        new_state = game.state.move(move)
        if new_state.winner == current_player:
            game.state = new_state
            return

    for move in available_moves:
        test_state = game.state.move(move)
        test_state.player = opponent
        for opp_move in test_state.available_moves:
            if opp_move == move:
                continue
            potential_state = test_state.move(opp_move)
            if potential_state.winner == opponent:
                game.state = game.state.move(move)
                return

    best_move = None
    best_score = float('-inf')
    for move in available_moves:
        new_state = game.state.move(move)
        score = (new_state.count_lines(3, current_player) * 10 + 
                 new_state.central(current_player) * 2 - 
                 new_state.count_lines(3, opponent) * 8)
        if score > best_score:
            best_score = score
            best_move = move

    if best_move is None:
        for preferred in [3, 2, 4, 1, 5, 0, 6]:
            if preferred in available_moves:
                best_move = preferred
                break

    game.state = game.state.move(best_move)

In [5]:
# === 5. Monte Carlo Tree Search ===
import math

class MCNode:
    def __init__(self, game, player):
        self.game = game
        self.visits = 1
        self.wins = 0
        self.children = []
        self.player = player

    def is_leaf(self):
        return len(self.children) == 0

    def add_child(self, player):
        if not self.game.state.available_moves:
            return self
        for move in self.game.state.available_moves:
            new_game = deepcopy(self.game)
            new_game.state = new_game.state.move(move)
            self.children.append(MCNode(new_game, player))
        return random.choice(self.children)

    def update_stats(self, value):
        if self.player == 1 and value == 2:
            value = 0
        elif self.player == 2 and value == 1:
            value = 0
        elif value == self.player:
            value = 1
        else:
            value = 0
        self.wins += value
        self.visits += 1

class MCTree:
    def __init__(self, root):
        self.root = root

    def expand(self, node, player_cur):
        return node.add_child(player_cur)

    def visualize(self, filename="mcts_tree"):
        dot = Digraph(comment='Monte Carlo Tree')
        def add_nodes(dot, node, parent_id=None, node_id_gen=[0]):
            node_id = str(node_id_gen[0])
            symbol_map = {0: ".", 1: "X", 2: "O"}
            state_lines = [" ".join(symbol_map[cell] for cell in row) for row in reversed(node.game.state.board)]
            state_repr = "\\n".join(state_lines)
            label = f"P{node.player}\\nV:{node.visits-1}\\nW:{node.wins}\\n{state_repr}"
            dot.node(node_id, label)
            if parent_id is not None:
                dot.edge(parent_id, node_id)
            current_id = node_id
            node_id_gen[0] += 1
            for child in node.children:
                add_nodes(dot, child, current_id, node_id_gen)
        add_nodes(dot, self.root)
        dot.render(filename, format='png', cleanup=True)

def monte_carlo(epochs, tree, c, player):
    for _ in range(epochs):
        visited = []
        player_cur = player
        node = tree.root
        visited.append(node)
        while not node.is_leaf():
            node = select(node, node.children, c, player_cur)
            visited.append(node)
            player_cur = 3 - player_cur
        new_child = tree.expand(node, player_cur)
        visited.append(new_child)
        value = roll_out(new_child)
        for node in visited:
            node.update_stats(value)
    return best_action(tree.root)

def select(node, children, c, player):
    best_value = -math.inf
    best_node = None
    for child in children:
        uct_value = (child.wins / child.visits) + c * math.sqrt(math.log(node.visits) / child.visits)
        if child.player != player:
            uct_value = -uct_value
        if uct_value > best_value:
            best_value = uct_value
            best_node = child
    return best_node

def best_action(root):
    return max(root.children, key=lambda child: child.wins / child.visits)

def roll_out(node):
    while node.game.state.winner == -1:
        execute_random_move(node.game)
    return node.game.state.winner

def execute_monte_carlo_move(epochs, c, player):
    def inner(game):
        root = MCNode(game, 3 - player)
        next_node = monte_carlo(epochs, MCTree(root), c, player)
        game.state = next_node.game.state
    return inner


In [7]:
# === 6. Main Game Loop ===
def human_player_move(game):
    while True:
        try:
            print(f"\nPlayer {game.state.player}'s turn")
            print(f"Available columns: {[col+1 for col in game.state.available_moves]}")
            column = int(input("Enter column number (1-7): ")) - 1
            if column in game.state.available_moves:
                game.state = game.state.move(column)
                break
            else:
                print("Invalid move! Try again.")
        except ValueError:
            print("Please enter a valid number.")

if __name__ == "__main__":
    while True:
        print_welcome()
        choice = input("Select game mode: ")

        if choice == "1":
            game = ConnectFourGame(human_player_move, human_player_move)
            game.start()
        elif choice == "2":
            player_choice = input("Play as Player 1 (X) or Player 2 (O)? (1/2): ")
            monte_carlo_ai = execute_monte_carlo_move(200, 1.14, 1 if player_choice == "2" else 2)
            game = ConnectFourGame(human_player_move, monte_carlo_ai) if player_choice == "1" else ConnectFourGame(monte_carlo_ai, human_player_move)
            game.start()
        elif choice == "3":
            game = ConnectFourGame(execute_monte_carlo_move(200, 1.14, 1), execute_monte_carlo_move(200, 1.14, 2))
            num_games = int(input("How many games to run? "))
            game.run_n_matches(num_games)
        elif choice == "4":
            print("Thanks for playing!")
            break
        else:
            print("Invalid choice. Try again.")



===== CONNECT 4 =====
1. Player vs Player
2. Player vs Computer
3. Computer vs Computer
4. Exit
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!

=== Elapsed time: 6 seconds ===
  Player 1: 3 victories
  Player 2: 0 victories
  Draws: 0 

===== CONNECT 4 =====
1. Player vs Player
2. Player vs Computer
3. Computer vs Computer
4. Exit
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 2 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!

=== Elapsed time: 47 seconds ===
  Player 1: 9 victories
  Player 2: 1 victories
  Draws: 0 

===== CONNECT 4 =====
1. Player vs Player
2. Player vs Computer
3. Computer vs Computer
4. Exit
Thanks for playing!
